# Doge

训练 [Wonderful Matrices](https://arxiv.org/abs/2412.11834) 论文中提出的 `Doge` 小型语言模型.
Doge在 Transformers 的框架基础上, 将序列变换部分的 `Multi-Head Attention` 替换为 `Dynamic Mask Attention`, 将状态变换部分的 `MLP` 替换为 `CDMoE` . 

![doge_architecture](../assets/doge_architecture.png)

## 预训练

### 下载预训练与微调数据集


预训练数据集, 我们选取了 `fineweb-edu-dedup` 高质量文本, `cosmopedia-v2` 合成指令数据集, 并补充 `python-edu` 与 `open-web-math` 来保证模型的代码与数学能力. 


> 请注意: 由于数据集过大, 至少需要 2TB 的存储空间.

In [ ]:
# 填写保存路径, 缓存路径和进程数
!python ./examples/pretrain/scripts/download_datasets.py --save_dir ./datasets --cache_dir ./cache --num_proc 1

### 预处理数据集


我们需要使用 `tokenizer` 将数据集转为模型可接受的 `input_ids` 与 `attention_mask`.
Doge 使用 `LlamaTokenizer` , 该 tokenizer 词表大小为 `32768` , 使用 `[INST]` 与 `[/INST]` 标记指令. 它还包括工具标记, 但是我们不会在这里使用它们.
像 cosmopedia-v2 这样的数据集就包括 `prompt` 与 `text` 两个字段, 我们就将他们标记为用户内容与助手内容.


```python
conversation = [
    {"role": "user", "content": prompt},
    {"role": "assistant", "content": text},
]
return tokenizer.apply_chat_template(conversation, tokenize=True, padding='max_length', truncation=True, max_length=MAX_LENGTH, return_dict=True)
```


当然你也可以自行加入一些指令提示.


```python
conversation = [
    {"role": "user", "content": "Who are you?"},
    {"role": "assistant", "content": "I am an AI assistant named `Doge`, I am a language model trained by `Shi Jingze` based on the `Doge` architecture, and my task is to provide appropriate answers and support to users based on their questions and requests."},
    {"role": "user", "content": prompt},
    {"role": "assistant", "content": text},
]
```

In [ ]:
# 填写数据集路径, 保存路径, 分词器路径, 样本数量, 最大长度和进程数
# NOTE: 我们只保留 160B tokens 的数据集, 比例为 fineweb-edu:cosmopedia-v2:python-edu:open-web-math = 7:2:0.5:0.5, 如果你需要训练更大的模型, 请自行增加数据集的规模
!python ./examples/pretrain/scripts/preprocess_datasets.py --datasets_dir ./datasets --save_dir ./datasets --tokenizer_path ./examples/tokenizer --train_examples 81920000 --test_examples 1000 --max_length 2048 --num_proc 16

### 合并数据集


我们将 fineweb-edu_tokenized, cosmopedia-v2, python-edu 和 open-web-math 数据集合并为 `pretrain` 数据集.
然后将它们打乱顺序 `seed=233` , 并拆分出来 `1,000` 个样本作为测试集.

In [ ]:
# 填写数据集路径, 保存路径, 样本数量和进程数
!python ./examples/pretrain/scripts/concatenate_datasets.py --datasets_dir ./datasets --save_dir ./datasets --train_examples 81920000 --test_examples 1000 --num_proc 16

### 配置模型参数


我们配置一个 `20M` 的小型模型, 进行训练测试.


| Model | Params | n_layers | d_model | d_ff | n_heads | n_exprets | n_expert_heads | n_expert_pre_head |
|---|---|---|---|---|---|---|---|---|
| Doge-20M | 20M | 4 | 256 | 1024 | 2 | - | - | - |
| Doge-MoE-20M | 20M | 4 | 256 | 1024 | 2 | 1024 | 2 | 4 |
| Doge-60M | 60M | 8 | 512 | 2048 | 4 | - | - | - |
| Doge-MoE-80M | 80M | 8 | 512 | 2048 | 4 | 2048 | 4 | 8 |
| Doge-160M | 160M | 12 | 768 | 3072 | 6 | - | - | - |
| Doge-MoE-230M | 230M | 12 | 768 | 3072 | 6 | 3072 | 6 | 12 |
| Doge-320M | 320M | 16 | 1024 | 4096 | 8 | - | - | - |
| Doge-MoE-500M | 500M | 16 | 1024 | 4096 | 8 | 4096 | 8 | 16 |

- n_layers 是模型的解码器层数
- d_model 是模型的隐藏层维度
- n_heads 是多头注意力头数 d_model // n_heads 最好保持在 64 以上


> `Doge-MoE` 模型可以继承 `Doge` 模型的密集激活参数, 并通过设置 `n_experts`, `n_expert_heads`, `n_expert_pre_head` 来增加稀疏激活的参数.

### 配置预训练超参数


| Params | tokens | num_train_epochs | per_epoch_max_steps | accumulate_steps | learning_rate | warmup_ratio | weight_decay | min_lr_rate |
|--------|--------|------------------|---------------------|------------------|---------------|--------------|--------------|-------------|
| 20M    | 5B     | 2                | 10,000              | 128              | 8e-4          | 0.1          | 0.01         | 0.1         |
| 60M    | 20B    | 2                | 20,000              | 256              | 6e-4          | 0.1          | 0.01         | 0.1         |
| 160M   | 80B    | 2                | 40,000              | 512              | 5e-4          | 0.1          | 0.01         | 0.1         |
| 320M   | 320B   | 2                | 80,000              | 1024             | 4e-4          | 0.1          | 0.01         | 0.1         |

### 预训练模型

In [ ]:
# 你需要指定配置文件路径, 日志路径, 输出路径和分词器路径, 如果需要的话, 还可以指定检查点继续训练 --resume_from_checkpoint
!python ./examples/pretrain/scripts/pretrain.py --config_path ./examples/pretrain/configs/Doge-20M.yaml --logging_dir ./logs --output_dir ./results --tokenizer_path ./examples/tokenizer --resume_from_checkpoint <path_to_checkpoint>

### 使用


在完成训练后, 我们可以使用 `Transformers` 的 `AutoModelForCausalLM` 加载模型, 并使用 `AutoTokenizer` 加载 `LlamaTokenizer` .

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("JingzeShi/Doge-20M")
model = AutoModelForCausalLM.from_pretrained("JingzeShi/Doge-20M", trust_remote_code=True)

In [ ]:
inputs = tokenizer("Hey how are you doing?", return_tensors="pt")

out = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.batch_decode(out))

## 微调

### 下载微调数据集


微调数据集, 我们选取了 `smoltalk` 合成指令数据集, 来进行监督微调.

In [ ]:
# 填写保存路径, 缓存路径和进程数
!python ./examples/finetune/scripts/download_datasets.py --save_dir ./datasets --cache_dir ./cache --num_proc 1

### 处理微调数据集


我们将微调数据集应用于`聊天模板`.

In [ ]:
# 填充数据集存放路径 数据集保存路径、分词器路径、进程数量 
!python ./examples/finetune/scripts/preprocess_datasets.py --datasets_dir ./datasets --save_dir ./datasets --tokenizer_path ./examples/tokenizer --num_proc 8

### 微调模型

In [ ]:
# 你需要指定预训练模型名称或者路径, 配置文件路径, 日志路径和输出路径, 如果需要的话, 还可以指定检查点继续训练 --resume_from_checkpoint
!python ./examples/finetune/scripts/sft.py --pretrained_model_name_or_path JingzeShi/Doge-20M --config_path ./examples/finetune/configs/Doge-20M-Instruct.yaml --logging_dir ./logs --output_dir ./results --resume_from_checkpoint <path_to_checkpoint>

### 使用

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig, TextStreamer

tokenizer = AutoTokenizer.from_pretrained("JingzeShi/Doge-20M-Instruct")
model = AutoModelForCausalLM.from_pretrained("JingzeShi/Doge-20M-Instruct", trust_remote_code=True)

In [12]:
generation_config = GenerationConfig(
      max_new_tokens=100, 
      use_cache=True, 
      do_sample=True, 
      temperature=0.8, 
      repetition_penalty=1.0
)
steamer = TextStreamer(
      tokenizer=tokenizer, 
      skip_prompt=True
)

In [ ]:
prompt = "Hi, how are you doing today?"

conversation = [
      {"role": "user", "content": prompt}
]
inputs = tokenizer.apply_chat_template(
    conversation=conversation,
    tokenize=True,
    return_tensors="pt",
)

outputs = model.generate(
    inputs, 
    tokenizer=tokenizer,
    generation_config=generation_config, 
    streamer=steamer
)

## 评估


我们先安装 `miniconda` .


```bash
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
bash Miniconda3-latest-Linux-x86_64.sh
```

然后创建评估环境.


```bash
conda create -n lighteval python=3.10.12 
conda activate lighteval
pip install lighteval[accelerate]
```

最后我们运行评估脚本.


如果你使用 Linux, 你可以运行以下命令.


```bash
bash ./examples/evaluate/eval_downstream_tasks.sh
```

如果你使用 Windows, 你可以运行以下命令.


```bash
. ./examples/evaluate/eval_downstream_tasks.ps1
```

> 注意: 脚本中的 MODEL 也可以填入保存的检查点路径, 只需要注册保存即可运行.